In [1]:
import numpy as np 
import pandas as pd
from nma import load_single_timeseries 
from nma import load_evs
from hdf5storage import savemat

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond': ['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['mental','rnd']}}



my_exp = 'WM'
subjects = np.loadtxt('subjects_list.txt', dtype='str')
nsubjects = len(subjects)
conditions = EXPERIMENTS[my_exp]['cond']
nconditions = len(conditions)
print(conditions)
mysubj = subjects[0]
data = load_single_timeseries(subject=mysubj, experiment=my_exp, bold_run='RL')
nscan = np.shape(data)[1]
subj_fmri = np.zeros((N_PARCELS,len(subjects)*nscan*2))
condition_index = np.zeros(len(subjects)*nscan*2)
subject_index = np.zeros(len(subjects)*nscan*2)

for s in range(len(subjects)):
    my_subj = subjects[s]
    data = load_single_timeseries(subject=my_subj, experiment=my_exp, bold_run='RL')
    for j in range(data.shape[0]):
	    data[j,:] = data[j,:] - np.mean(data[j,:]) 
    evs = load_evs(subject=my_subj, experiment=my_exp, conditions = conditions,bold_run='RL')
    scan_index = dict()
    for j in range(len(evs)):
        x = evs[j]
        y = np.array(x[0])
        for k in range(1,len(x)):
            y = np.append(y,x[k])
        scan_index[conditions[j]] = y
    subj_fmri[:,s*2*nscan:(s*2+1)*nscan]= data
    trial_c_index = np.zeros(nscan)
    for j in range(nconditions):
        trial_c_index[scan_index[conditions[j]]] = j
    condition_index[s*2*nscan:(s*2+1)*nscan] = trial_c_index
    subject_index[s*2*nscan:(s*2+1)*nscan] = s

    data = load_single_timeseries(subject=my_subj,experiment=my_exp,bold_run='LR')
    for j in range(data.shape[0]):
	    data[j,:] = data[j,:] - np.mean(data[j,:])
    evs = load_evs(subject=my_subj, experiment=my_exp, conditions = conditions,bold_run='LR')
    scan_index = dict()
    for j in range(len(evs)):
        x = evs[j]
        y = np.array(x[0])
        for k in range(1,len(x)):
            y = np.append(y,x[k])
        scan_index[conditions[j]] = y
    subj_fmri[:,(s*2+1)*nscan:(s*2+2)*nscan]= data
    trial_c_index = np.zeros(nscan)
    for j in range(nconditions):
        trial_c_index[scan_index[conditions[j]]] = j
    condition_index[(s*2+1)*nscan:(s*2+2)*nscan] = trial_c_index
    subject_index[(s*2+1)*nscan:(s*2+2)*nscan] = s

#z_lf = np.mean(data[:,scan_index['lf']],axis = 1)
#z_rf = np.mean(data[:,scan_index['rf']],axis = 1)


['0bk_body', '0bk_faces', '0bk_places', '0bk_tools', '2bk_body', '2bk_faces', '2bk_places', '2bk_tools']


In [2]:
fmri = dict()
fmri['fmri'] = subj_fmri
fmri['subject'] = subject_index
fmri['condition_index'] = condition_index
fmri['conditions'] = conditions
fmri['task'] = my_exp
fmri['nsubjects'] = nsubjects
fmri['nconditions'] = nconditions
fmri['nregions'] = N_PARCELS
fmri['nscan'] = nscan

savemat(my_exp+'_fmri.mat',fmri,store_python_metadata=True)